In [1]:
%matplotlib inline
import IPython.display
from ipywidgets import interact, interactive, fixed
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib as a
import copy
from scipy.io import wavfile
from scipy.signal import butter, lfilter
import scipy.ndimage
import pylab
import tensorflow as tf
import scipy.misc as mi
from PIL import Image

In [2]:
# Most of the Spectrograms and Inversion are taken from: https://gist.github.com/kastnerkyle/179d6e9a88202ab0a2fe
# This function is used to pass the sound data through a filter to limit the low and the high frequency.

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def overlap(X, window_size, window_step):
    """
    Create an overlapped version of X
    Parameters
    ----------
    X : ndarray, shape=(n_samples,)
        Input signal to window and overlap
    window_size : int
        Size of windows to take
    window_step : int
        Step size between windows
    Returns
    -------
    X_strided : shape=(n_windows, window_size)
        2D array of overlapped X
    """
    if window_size % 2 != 0:
        raise ValueError("Window size must be even!")
    # Make sure there are an even number of windows before stridetricks
    append = np.zeros((window_size - len(X) % window_size))
    X = np.hstack((X, append))
    print ('X is', X)

    #ws = window_size
    #ss = window_step
    ws = window_size
    ss = int(window_step)
    a = X
    #print('window_step: ',window_step)
    #print('window_size: ',window_size)

    valid = len(a) - ws
    nw = int((valid) // ss)
    #print(nw)
    #print('Nw is', nw)
    out = np.ndarray((nw,ws),dtype = a.dtype)

    #for i in xrange(nw):
    for i in range(nw):
        # "slide" the window along the samples
        start = i * ss
        #print('Start is', start)
        stop = start + ws
        #print ('stop is', stop)
        out[i] = a[start : stop]

    return out


def stft(X, fftsize=128, step=65, mean_normalize=True, real=False,
         compute_onesided=True):
    """
    Compute STFT for 1D real valued input X
    """
    if real:
        local_fft = np.fft.rfft
        cut = -1
    else:
        local_fft = np.fft.fft
        cut = None
    if compute_onesided:
        cut = fftsize // 2
    if mean_normalize:
        X -= X.mean()

    X = overlap(X, fftsize, step)
    
    size = fftsize
    win = 0.54 - .46 * np.cos(2 * np.pi * np.arange(size) / (size - 1))
    X = X * win[None]
    X = local_fft(X)[:, :cut]
    return X

def pretty_spectrogram(d,log = True, thresh= 5, fft_size = 512, step_size = 64):
    """
    creates a spectrogram
    log: take the log of the spectrgram
    thresh: threshold minimum power for log spectrogram
    """
    specgram = np.abs(stft(d, fftsize=fft_size, step=step_size, real=False,
        compute_onesided=True))
  
    if log == True:
        specgram /= specgram.max() # volume normalize to max 1
        specgram = np.log10(specgram) # take log
        specgram[specgram < -thresh] = -thresh # set anything less than the threshold as the threshold
    else:
        specgram[specgram < thresh] = thresh # set anything less than the threshold as the threshold
    
    return specgram

# Also mostly modified or taken from https://gist.github.com/kastnerkyle/179d6e9a88202ab0a2fe
def invert_pretty_spectrogram(X_s, log = True, fft_size = 512, step_size = 512/4, n_iter = 10):
    
    if log == True:
        X_s = np.power(10, X_s)

    X_s = np.concatenate([X_s, X_s[:, ::-1]], axis=1)
    X_t = iterate_invert_spectrogram(X_s, fft_size, step_size, n_iter=n_iter)
    return X_t

def iterate_invert_spectrogram(X_s, fftsize, step, n_iter=10, verbose=False):
    """
    Under MSR-LA License
    Based on MATLAB implementation from Spectrogram Inversion Toolbox
    References
    ----------
    D. Griffin and J. Lim. Signal estimation from modified
    short-time Fourier transform. IEEE Trans. Acoust. Speech
    Signal Process., 32(2):236-243, 1984.
    Malcolm Slaney, Daniel Naar and Richard F. Lyon. Auditory
    Model Inversion for Sound Separation. Proc. IEEE-ICASSP,
    Adelaide, 1994, II.77-80.
    Xinglei Zhu, G. Beauregard, L. Wyse. Real-Time Signal
    Estimation from Modified Short-Time Fourier Transform
    Magnitude Spectra. IEEE Transactions on Audio Speech and
    Language Processing, 08/2007.
    """
    reg = np.max(X_s) / 1E8
    X_best = copy.deepcopy(X_s)
    for i in range(n_iter):
        if verbose:
            print("Runnning iter %i" % i)
        if i == 0:
            X_t = invert_spectrogram(X_best, step, calculate_offset=True,
                                     set_zero_phase=True)
        else:
            # Calculate offset was False in the MATLAB version
            # but in mine it massively improves the result
            # Possible bug in my impl?
            X_t = invert_spectrogram(X_best, step, calculate_offset=True,
                                     set_zero_phase=False)
        est = stft(X_t, fftsize=fftsize, step=step, compute_onesided=False)
        phase = est / np.maximum(reg, np.abs(est))
        X_best = X_s * phase[:len(X_s)]
    X_t = invert_spectrogram(X_best, step, calculate_offset=True,
                             set_zero_phase=False)
    return np.real(X_t)

def invert_spectrogram(X_s, step, calculate_offset=True, set_zero_phase=True):
    """
    Under MSR-LA License
    Based on MATLAB implementation from Spectrogram Inversion Toolbox
    References
    ----------
    D. Griffin and J. Lim. Signal estimation from modified
    short-time Fourier transform. IEEE Trans. Acoust. Speech
    Signal Process., 32(2):236-243, 1984.
    Malcolm Slaney, Daniel Naar and Richard F. Lyon. Auditory
    Model Inversion for Sound Separation. Proc. IEEE-ICASSP,
    Adelaide, 1994, II.77-80.
    Xinglei Zhu, G. Beauregard, L. Wyse. Real-Time Signal
    Estimation from Modified Short-Time Fourier Transform
    Magnitude Spectra. IEEE Transactions on Audio Speech and
    Language Processing, 08/2007.
    """
    size = int(X_s.shape[1] // 2)
    step=int(step)
    wave = np.zeros((X_s.shape[0] * step + size))
    # Getting overflow warnings with 32 bit...
    wave = wave.astype('float64')
    total_windowing_sum = np.zeros((X_s.shape[0] * step + size))
    win = 0.54 - .46 * np.cos(2 * np.pi * np.arange(size) / (size - 1))

    est_start = int(size // 2) - 1
    est_end = est_start + size
    for i in range(X_s.shape[0]):
        wave_start = int(step * i)
        wave_end = wave_start + size
        if set_zero_phase:
            spectral_slice = X_s[i].real + 0j
        else:
            # already complex
            spectral_slice = X_s[i]

        # Don't need fftshift due to different impl.
        wave_est = np.real(np.fft.ifft(spectral_slice))[::-1]
        if calculate_offset and i > 0:
            offset_size = size - step
            if offset_size <= 0:
                print("WARNING: Large step size >50\% detected! "
                      "This code works best with high overlap - try "
                      "with 75% or greater")
                offset_size = step
            offset = xcorr_offset(wave[wave_start:wave_start + offset_size],
                                  wave_est[est_start:est_start + offset_size])
        else:
            offset = 0
        
        wave = wave.astype('float64')
        wave[wave_start:wave_end] += win * wave_est[
            est_start - offset:est_end - offset]
        total_windowing_sum[wave_start:wave_end] += win
    wave = np.real(wave) / (total_windowing_sum + 1E-6)
    return wave

def xcorr_offset(x1, x2):
    """
    Under MSR-LA License
    Based on MATLAB implementation from Spectrogram Inversion Toolbox
    References
    ----------
    D. Griffin and J. Lim. Signal estimation from modified
    short-time Fourier transform. IEEE Trans. Acoust. Speech
    Signal Process., 32(2):236-243, 1984.
    Malcolm Slaney, Daniel Naar and Richard F. Lyon. Auditory
    Model Inversion for Sound Separation. Proc. IEEE-ICASSP,
    Adelaide, 1994, II.77-80.
    Xinglei Zhu, G. Beauregard, L. Wyse. Real-Time Signal
    Estimation from Modified Short-Time Fourier Transform
    Magnitude Spectra. IEEE Transactions on Audio Speech and
    Language Processing, 08/2007.
    """
    x1 = x1 - x1.mean()
    x2 = x2 - x2.mean()
    frame_size = len(x2)
    half = frame_size // 2
    corrs = np.convolve(x1.astype('float32'), x2[::-1].astype('float32'))
    corrs[:half] = -1E30
    corrs[-half:] = -1E30
    offset = corrs.argmax() - len(x1)
    return offset

def make_mel(spectrogram, mel_filter, shorten_factor = 1):
    mel_spec =np.transpose(mel_filter).dot(np.transpose(spectrogram))
    mel_spec = scipy.ndimage.zoom(mel_spec.astype('float32'), [1, 1./shorten_factor]).astype('float16')
    mel_spec = mel_spec[:,1:-1] # a little hacky but seemingly needed for clipping 
    return mel_spec

def mel_to_spectrogram(mel_spec, mel_inversion_filter, spec_thresh, shorten_factor):
    """
    takes in an mel spectrogram and returns a normal spectrogram for inversion 
    """
    mel_spec = (mel_spec+spec_thresh)
    uncompressed_spec = np.transpose(np.transpose(mel_spec).dot(mel_inversion_filter))
    uncompressed_spec = scipy.ndimage.zoom(uncompressed_spec.astype('float32'), [1,shorten_factor]).astype('float16')
    uncompressed_spec = uncompressed_spec -4
    return uncompressed_spec

In [3]:
### Parameters ###
fft_size = 2048 # window size for the FFT
step_size = fft_size/16 # distance to slide along the window (in time)
spec_thresh = 4 # threshold for spectrograms (lower filters out more noise)
lowcut = 500 # Hz # Low cut for our butter bandpass filter
highcut = 15000 # Hz # High cut for our butter bandpass filter
# For mels
n_mel_freq_components = 64 # number of mel frequency channels
shorten_factor = 10 # how much should we compress the x-axis (time)
start_freq = 300 # Hz # What frequency to start sampling our melS from 
end_freq = 8000 # Hz # What frequency to stop sampling our melS from

In [4]:
# Reading the Music file. Grab your wav and filter it
mywav = 'gun_battle_sound-ReamProductions-1158375208.wav'
# Reading the data and the frequency rate.
rate, data = wavfile.read(mywav)
print('Rate:', rate)
#####################################################################################
# Finding the number of seconds in the data file
seconds = int(data[:,1].shape[0]/rate)
print('Total Seconds:', seconds)
# Passing the data through a band pass filter to filter the low cut and high cut frequencies. The lowcut frequency is 
# set at 500 Hz and the high cut is set at 15000 Hz
data = butter_bandpass_filter(data, lowcut, highcut, rate, order=1)
# Only use a short clip for our demo
# Clip the data into 2 seconds.
if np.shape(data)[0]/float(rate) > 1:
    data = data[0:rate*1] 
print ('Length in time (s): ', np.shape(data)[0]/float(rate))
# Extracting the frequency data from the music data. This is the data we are going to save into images.
data=data[:,1]

C:\Users\sedkilic\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\io\wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


Rate: 44100
Total Seconds: 20
Length in time (s):  1.0


In [5]:
# Finding the minimim and maximum of the data.
vmin=data.min() 
vmax=data.max()
norm = plt.Normalize(vmin=data.min(), vmax=data.max())
# Spilting the data into 20 samples each of one second
samples=np.array_split(data, seconds-1)
i=0
#Generating Greyscale Images for Each Second and saving them as grey + i.png
for x in samples:
    i=i+1
    filename='Gun/grey'+str(i)+'.png'
    img=mi.toimage(data.reshape(210,210), cmin=x.min(), cmax=x.max())
    #The commented code below to scale with min and max frequency of the whole sound file. This is not necessary in this approach
    #as we are training separately on for image generated for each second of music.
    #img=mi.toimage(x.reshape(210,210), cmin=vmin, cmax=vmax)
    img.save(filename)
    #unscaled = np.reshape(img, (1,np.product([210,210])))
    #reshaped=unscaled.reshape(44100)
    #print(reshaped)
print('Sample Music:')
# Playing the sample music of 9 seconds
IPython.display.Audio(data=data, rate=rate)

C:\Users\sedkilic\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  if sys.path[0] == '':


Sample Music:


In [6]:
# Playing the first second of sample music.
print('first second of sample music')
IPython.display.Audio(data=samples[0], rate=rate)

first second of sample music


In [8]:
# Playing the 2nd second of sample music
print('2nd second of sample music')
IPython.display.Audio(data=samples[1], rate=rate)

2nd second of sample music


In [9]:
# # Playing the 3rd second of sample music
print('Sample Music third second')
IPython.display.Audio(data=samples[2], rate=rate)

Sample Music third second


In [22]:
img1 = mi.imread('08700.png')
unscaled1 = np.reshape(img1, (1,np.product([160,160])))
reshaped1=unscaled1.reshape(25600)
print(reshaped1)

wav_spectrogram1 = pretty_spectrogram(reshaped1.astype('float64'), fft_size = fft_size, 
                                   step_size = step_size, log = True, thresh = spec_thresh)
print('Generated audio for 1st second')
# Playing the sound generated for second 1
IPython.display.Audio(data=reshaped1, rate=rate)

[146 142 166 ...   9  10 165]
X is [17.51964844 13.51964844 37.51964844 ...  0.          0.
  0.        ]
Generated audio for 1st second


C:\Users\sedkilic\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  """Entry point for launching an IPython kernel.


In [11]:
img1 = mi.imread('00000.png')
np.shape(img1)

C:\Users\sedkilic\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  """Entry point for launching an IPython kernel.


(160, 160)

In [12]:
# Saving Sound generated from image for second 1
scipy.io.wavfile.write('second1_recovered.wav', 44100, reshaped1)

In [ ]:
img2 = mi.imread('00000.png')
unscaled2 = np.reshape(img2, (1,np.product([210,210])))
reshaped2=unscaled2.reshape(44100)
wav_spectrogram2 = pretty_spectrogram(reshaped2.astype('float64'), fft_size = fft_size, 
                                   step_size = step_size, log = True, thresh = spec_thresh)

print('Generated audio for 2nd second')
# Playing the sound generated for second 2
IPython.display.Audio(data=reshaped2, rate=rate)

In [99]:
# Saving Sound generated from image for second 2
scipy.io.wavfile.write('second2_recovered.wav', 44100, reshaped2)

In [ ]:
img3 = mi.imread('grey8.png')
unscaled3 = np.reshape(img3, (1,np.product([210,210])))
reshaped3=unscaled3.reshape(44100)

wav_spectrogram3 = pretty_spectrogram(reshaped3.astype('float64'), fft_size = fft_size, 
                                   step_size = step_size, log = True, thresh = spec_thresh)
print('Generated audio for 3rd second')
# Playing the sound generated from image 3.
IPython.display.Audio(data=reshaped3, rate=rate)

In [103]:
# Saving Sound generated from image for second 3
scipy.io.wavfile.write('second3_recovered.wav', 44100, reshaped3)

In [105]:
app=np.concatenate((reshaped1,reshaped2))
app=np.concatenate((app,reshaped3))
print('Generated Audio Combined for 4 seconds.')
# Playing the combined sound.
IPython.display.Audio(data=app, rate=rate)

Generated Audio Combined for 4 seconds.
